In [1]:
%matplotlib inline
import re
import seaborn as sbn
import nltk
import tqdm as tqdm
import sqlite3
import pandas as pd
import numpy as np
nltk.download('stopwords')


import string
from nltk.corpus import stopwords
stop = stopwords.words("english")
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from math import floor,ceil
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

from sklearn.svm import LinearSVC

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sujaykumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('../data/software_text.csv')

In [3]:
df = df.drop(['Unnamed: 0','style','vote','image'],axis=1)

<h2>As we are taking only verified Users we drop all the false Reviews</h2>

In [4]:
df["verified"].value_counts()

False    7631
True     5174
Name: verified, dtype: int64

In [5]:
df = df[df['verified']]

In [6]:
df["verified"].value_counts()

True    5174
Name: verified, dtype: int64

<h3>We Consider only the Text Based to Recommend the Products</h3>

In [7]:
df = df.drop(['reviewTime','reviewerName','unixReviewTime','verified'],axis=1)

In [8]:
df = df.dropna(axis=0)

In [9]:
df

,overall,reviewerID,asin,reviewText,summary
22,5.0,AHEZP9ACDYJII,0763855553,Microsoft Office is still the standard all the...,Still the Standard
23,1.0,A2WUYJJ7NPQ5H4,0763855553,Been using Office for over twenty years. Stil...,and easily corrupted. Outlook is all messed up
24,5.0,AXGLPF6X39874,0763855553,Always liked Office Products. Expensive for a...,Best value
25,5.0,AJK49DENMS2BT,0763855553,I have been using MS Office professional for d...,despite competing support elsewhere in the sof...
26,5.0,A3UI63EESKUYHU,0763855553,This is really a great buy. It breaks down to...,Excellent value
...,...,...,...,...,...
12792,5.0,A3PGN4ZXMQRSYH,B01DEG0SGC,Arrived as described. I was pleased with purc...,I was pleased with
12793,1.0,A2DKUVUY6M48YF,B01E6C5D1S,Was not what I expected,Was not what I expected
12794,4.0,A1MOLMEBRO23PR,B01E6C5D1S,great buy,Four Stars
12795,4.0,A1RHJX6OA0O9KQ,B01E6C5D1S,"The product itself is quite good, in depth fix...",They put out a good product...all they need is...


In [10]:
df['overall'].value_counts()

5.0    3250
4.0     837
3.0     447
1.0     428
2.0     206
Name: overall, dtype: int64

<h3> Cleaning the Data (NLP)</h3>

In [11]:
def data_clean(rev, remove_stopwords=True):
    if isinstance(rev, str):
        new_text = re.sub("[^a-zA-Z]", " ", rev)
        words = new_text.lower().split()
        if remove_stopwords:
            sts = set(stopwords.words("english"))
            words = [w for w in words if not w in sts]
        ary = []
        eng_stemmer = english_stemmer
        for word in words:
            ary.append(eng_stemmer.stem(word))
        return ary
    else:
        return []

In [12]:
clean_reviewData = []
for rev in df['reviewText']:
    clean_reviewData.append( " ".join(data_clean(rev)))

clean_summaryData = []
for rev in df['summary']:
    clean_summaryData.append( " ".join(data_clean(rev)))

In [13]:
Most_used_Words_Review = pd.Series(' '.join(clean_reviewData).lower().split()).value_counts()[:20]
print(Most_used_Words_Review)

use        3736
year       2419
work       2211
softwar    1907
product    1837
instal     1776
tax        1703
window     1637
program    1575
version    1506
one        1459
get        1333
time       1292
like       1252
comput     1197
good       1173
need       1160
great      1071
file       1056
problem     964
dtype: int64


In [14]:
Most_used_Words_Summary = pd.Series(' '.join(clean_summaryData).lower().split()).value_counts()[:20]
print (Most_used_Words_Summary)

star        1223
five         789
great        482
work         422
good         406
product      307
tax          278
softwar      244
use          243
four         201
best         194
year         162
easi         160
excel        138
price        128
one          127
program      125
norton       112
better       108
turbotax     105
dtype: int64


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_vectorizer = TfidfVectorizer(min_df=4, max_features = 1000)
test_vecor = text_vectorizer.fit_transform(clean_reviewData)
tfidf_vector = dict(zip(text_vectorizer.get_feature_names_out(), text_vectorizer.idf_))

In [16]:
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
sample_review = df.reviewText[:10]
for test in sample_review:
    test
    ss = analyser.polarity_scores(test)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]))
    print(test)

compound: 0.4767, 
neg: 0.056, 
neu: 0.806, 
pos: 0.138, 
Microsoft Office is still the standard all the rest are compared to.  Office 365 is a great value when you consider the licensing options for standalone packages.  You might be tempted to use a free online office solution, but you'll be disappointed.  The cost and licensing options for this software are a bargain compared to the cost of owning the package on the same machines, and keeping them upgraded.
compound: 0.8302, 
neg: 0.03, 
neu: 0.901, 
pos: 0.07, 
Been using Office for over twenty years.  Still not worth the money.  Buggy, and easily corrupted. Outlook is all messed up. This is fourth year on 365.  Thought I could just click "renew" somewhere, pay, and be done with it.  Took for ever to finally find the right Office site, tried to renew, but couldn't as I didn't buy originally from there.  So I came here, bought it, and spent a good bit of time trying to figure out where to enter the key.  Ended up being where I could

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sujaykumar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/var/folders/l2/_mks1sd56yjdxq9514qt1fnh0000gn/T/ipykernel_12058/3928246299.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  sample_review = df.reviewText[:10]


In [17]:
test_reviewText = df.reviewText
test_Ratings = df.overall
text_vectorizer = TfidfVectorizer(max_df=.8)
text_vectorizer.fit(test_reviewText)
def rate(r):
    ary2 = []
    for rating in r:
        tv = [0,0,0,0,0]
        tv[rating-1] = 1
        ary2.append(tv)
    return np.array(ary2)

In [18]:
test_Ratings = df.overall

In [19]:
test_reviewText

22       Microsoft Office is still the standard all the...
23       Been using Office for over twenty years.  Stil...
24       Always liked Office Products.  Expensive for a...
25       I have been using MS Office professional for d...
26       This is really a great buy.  It breaks down to...
                               ...                        
12792    Arrived as described.  I was pleased with purc...
12793                              Was not what I expected
12794                                            great buy
12795    The product itself is quite good, in depth fix...
12796                                                   ok
Name: reviewText, Length: 5168, dtype: object

In [32]:
test_Ratings
rate = []
for i in test_Ratings:
    if i >=3:
        rate.append(1)
    else:
        rate.append(0)

In [36]:
test_Ratings

22       5.0
23       1.0
24       5.0
25       5.0
26       5.0
        ... 
12792    5.0
12793    1.0
12794    4.0
12795    4.0
12796    4.0
Name: overall, Length: 5168, dtype: float64

In [34]:
df = pd.DataFrame({
    'reviewText': test_reviewText,
    'Ratings': rate
})

In [35]:
df.to_csv("processed_data/final_pre-processed.csv")

In [37]:
df1 = pd.read_csv('../data/software_text.csv')

In [39]:
df1["verified"].value_counts()

False    7631
True     5174
Name: verified, dtype: int64

In [40]:
df1

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,0,4.0,False,"10 20, 2010",A38NELQT98S4H8,0321719816,{'Format:': ' DVD-ROM'},WB Halper,I've been using Dreamweaver (and it's predeces...,A solid overview of Dreamweaver CS5,1287532800,NaN,NaN
1,1,4.0,False,"10 18, 2010",A3QJU4FEN8PQSZ,0321719816,{'Format:': ' DVD-ROM'},Grimmy,"The demo is done with the PC version, with ref...",A good value,1287360000,NaN,NaN
2,2,5.0,False,"10 16, 2010",ACJT8MUC0LRF0,0321719816,{'Format:': ' DVD-ROM'},D. Fowler,If you've been wanting to learn how to create ...,This is excellent software for those who want ...,1287187200,3,NaN
3,3,5.0,False,"10 12, 2010",AYUF7YETYOLNX,0321719816,{'Format:': ' DVD-ROM'},Bryan Newman,I've been creating websites with Dreamweaver f...,A Fantastic Overview of Dream Weaver and Web D...,1286841600,NaN,NaN
4,4,5.0,False,"10 7, 2010",A31ICLWQ9CSHRS,0321719816,{'Format:': ' DVD-ROM'},Al Swanson,I decided (after trying a number of other prod...,Excellent Tutorials!,1286409600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12800,12800,4.0,False,"07 16, 2016",A1E50L7PCVXLN4,B01FFVDY9M,{'Platform:': ' Key Card'},Colinda,When I ordered this it was listed as Photo Edi...,File Management Software with Basic Editing Ca...,1468627200,NaN,NaN
12801,12801,3.0,False,"06 17, 2017",AVU1ILDDYW301,B01HAP3NUG,NaN,G. Hearn,This software has SO much going on. Theres a ...,"Might not be for the ""novice""",1497657600,NaN,NaN
12802,12802,4.0,False,"01 24, 2017",A2LW5AL0KQ9P1M,B01HAP3NUG,NaN,Dr. E,I have used both more complex and less complex...,"Great, Inexpensive Software for Those Who Have...",1485216000,NaN,NaN
12803,12803,3.0,False,"06 14, 2018",AZ515FFZ7I2P7,B01HAP47PQ,{'Platform:': ' PC Disc'},Jerry Jackson Jr.,Pinnacle Studio 20 Ultimate is a perfectly ser...,Gets the job done ... but not as easy as it sh...,1528934400,NaN,NaN


In [43]:
df1['style'].value_counts()

{'Platform:': ' PC Disc'}        1394
{'Platform:': ' PC Download'}     693
{'Format:': ' Software'}          668
{'Format:': ' CD-ROM'}            630
{'Platform:': ' PC/Mac Disc'}     494
                                 ... 
{'Color:': ' 0.0385987943'}         1
{'Color:': ' 0.5087989573'}         1
{'Color:': ' 0.4882057053'}         1
{'Color:': ' 0.2551958794'}         1
{'Color:': ' 0.0811504547'}         1
Name: style, Length: 238, dtype: int64